# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

False

In [ ]:
#| export
import re
import json
import time

from lisette import *
from toolslm.funccall import mk_ns, call_func
from litellm import completion

In [ ]:
#| export
def advanced_toolloop(message, sp, tools, sh=None, model='gpt-4', base_url=None, max_steps=10, 
                      final_prompt=None,
                      verbose=False
                     ):
    """
    Advanced tool loop for RLM that handles REPL code execution and recursive LLM calls.
    
    Args:
        message: User query to answer
        sp: System prompt for the root LLM
        tools: List of tool functions (typically [run_repl])
        sh: Shell instance for FINAL_VAR variable lookup
        model: Model name
        base_url: API base URL
        max_steps: Maximum iteration steps
        final_prompt: Prompt to add if max_steps reached without FINAL
    Yields:
        LLM responses, tool results, and final answer dict
    """
    if final_prompt is None:
        final_prompt = "Summarize your findings based on the tool results."
    
    ns = mk_ns(tools)
    tool_schemas = [lite_mk_func(t) for t in tools]

    if sp is not None:
        messages = [{'role': 'system', 'content': sp}, 
                    {'role': 'user', 'content': message}]
    else:
        messages = [{'role': 'user', 'content': message}]
    
    final_found = False
        
    for step in range(1, max_steps + 1):
        if verbose:
            print(f"[RLM] Step: {step}/{max_steps}")
            
        is_final = (step == max_steps)
        tool_choice = 'none' if is_final else None
        
        kwargs = {"model": model, "messages": messages, "tools": tool_schemas, "tool_choice": tool_choice}
        if base_url:
            kwargs["api_base"] = base_url
        
        r = completion(**kwargs)
        yield r
        
        msg = r.choices[0].message
        messages.append(msg)

        if msg.content:
            final_match = re.search(r'FINAL\((.*?)\)', msg.content, re.DOTALL)
            final_var_match = re.search(r'FINAL_VAR\(([^)]+)\)', msg.content)
    
            if final_match:
                if verbose:
                    print(f"[RLM] FINAL answer found")
                
                answer = final_match.group(1).strip()
                yield {"type": "final", "answer": answer}
                final_found = True
                break
            elif final_var_match:
                if verbose:
                    print(f"[RLM] FINAL answer found")
                
                var_name = final_var_match.group(1).strip()
                answer = sh.user_ns.get(var_name, f"Variable {var_name} not found")
                yield {"type": "final", "answer": str(answer)}
                final_found = True
                break

        if tool_calls := msg.tool_calls:
            for tc in tool_calls:
                if verbose:
                    print(f"  → Tool: {tc.function.name}")
                
                args = json.loads(tc.function.arguments)
                result = call_func(tc.function.name, args, ns=ns)
                tool_result = {
                    "tool_call_id": tc.id,
                    "role": "tool",
                    "name": tc.function.name,
                    "content": str(result)
                }
                messages.append(tool_result)
                yield tool_result
            
            if is_final:
                messages.append({'role': 'user', 'content': final_prompt})
        else:
            break
    
    # Fallback: if no FINAL() was found, use last assistant message
    if not final_found:
        if verbose:
            print(f"[RLM] Using fallback: no FINAL() detected")
        for msg in reversed(messages):
            if isinstance(msg, dict) and msg.get('role') == 'assistant':
                yield {"type": "final", "answer": msg.get('content', 'No answer provided')}
                break
            elif hasattr(msg, 'content') and msg.content:
                yield {"type": "final", "answer": msg.content}
                break

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()